# Drugi dio projekta - replikacija rezultata iz članka

In [548]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif

Učitajmo najprije podatke potrebne za rad.

In [549]:
teams = pd.read_csv("./data/proj2/teams.csv")
seasons = pd.read_csv("./data/proj2/seasons.csv")

season_compact_results = pd.read_csv("./data/proj2/regular_season_compact_results.csv")
season_detailed_results = pd.read_csv("./data/proj2/regular_season_detailed_results.csv")

tourney_compact_results = pd.read_csv("./data/proj2/tourney_compact_results.csv")
tourney_detailed_results = pd.read_csv("./data/proj2/tourney_detailed_results.csv")


Uklonimo zatim podatke koji nam neće trebati (sezone koje nisu uzete u obzir prilikom izvođenja rada).

In [550]:
seasons = seasons[(seasons.season == 2011) | (seasons.season == 2012) | (seasons.season == 2013) | (seasons.season == 2014)]

seasons

,season,dayzero,regionW,regionX,regionY,regionZ
26,2011,11/01/2010,East,West,Southeast,Southwest
27,2012,10/31/2011,East,Midwest,South,West
28,2013,11/05/2012,East,South,Midwest,West
29,2014,11/04/2013,East,South,Midwest,West


In [551]:
teams

,team_id,team_name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
...,...,...
359,1460,Wright St
360,1461,Wyoming
361,1462,Xavier
362,1463,Yale


In [552]:
season_compact_results = season_compact_results[(season_compact_results.season == 2011) 
                                                | (season_compact_results.season == 2012)
                                                | (season_compact_results.season == 2013)
                                                | (season_compact_results.season == 2014)]

season_compact_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
113385,2011,7,1228,79,1414,65,H,0
113386,2011,7,1268,105,1370,76,H,0
113387,2011,7,1338,83,1348,75,H,0
113388,2011,7,1400,83,1298,52,H,0
113389,2011,9,1228,84,1405,45,H,0
...,...,...,...,...,...,...,...,...
134561,2014,132,1196,61,1246,60,N,0
134562,2014,132,1277,69,1276,55,N,0
134563,2014,132,1386,65,1433,61,N,0
134564,2014,132,1418,82,1209,81,N,1


In [553]:
season_detailed_results = season_detailed_results[(season_detailed_results.season == 2011) 
                                                | (season_detailed_results.season == 2012)
                                                | (season_detailed_results.season == 2013)
                                                | (season_detailed_results.season == 2014)]

season_detailed_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
39337,2011,7,1228,79,1414,65,H,0,29,58,...,20,14,22,11,26,12,14,7,1,21
39338,2011,7,1268,105,1370,76,H,0,35,62,...,16,17,22,18,16,11,27,18,1,31
39339,2011,7,1338,83,1348,75,H,0,29,64,...,32,13,16,10,21,16,17,7,6,25
39340,2011,7,1400,83,1298,52,H,0,30,59,...,23,10,18,8,22,8,16,5,5,24
39341,2011,9,1228,84,1405,45,H,0,37,64,...,12,10,19,6,17,9,23,6,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60513,2014,132,1196,61,1246,60,N,0,23,53,...,11,21,26,12,24,9,12,4,8,19
60514,2014,132,1277,69,1276,55,N,0,27,54,...,23,15,16,13,15,7,9,3,3,15
60515,2014,132,1386,65,1433,61,N,0,24,50,...,19,8,15,13,25,9,5,6,6,18
60516,2014,132,1418,82,1209,81,N,1,32,75,...,19,20,21,4,26,8,7,5,7,15


In [554]:
tourney_compact_results = tourney_compact_results[(tourney_compact_results.season == 2011) 
                                                | (tourney_compact_results.season == 2012)
                                                | (tourney_compact_results.season == 2013)]

tourney_compact_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
1648,2011,134,1155,70,1412,52,N,0
1649,2011,134,1421,81,1114,77,N,1
1650,2011,135,1427,70,1106,61,N,0
1651,2011,135,1433,59,1425,46,N,0
1652,2011,136,1139,60,1330,58,N,0
...,...,...,...,...,...,...,...,...
1844,2013,146,1257,85,1181,63,N,0
1845,2013,146,1276,79,1196,59,N,0
1846,2013,152,1257,72,1455,68,N,0
1847,2013,152,1276,61,1393,56,N,0


In [555]:
tourney_detailed_results = tourney_detailed_results[(tourney_detailed_results.season == 2011) 
                                                | (tourney_detailed_results.season == 2012)
                                                | (tourney_detailed_results.season == 2013)]

tourney_detailed_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
512,2011,134,1155,70,1412,52,N,0,26,50,...,24,4,7,7,22,10,19,4,3,10
513,2011,134,1421,81,1114,77,N,1,27,54,...,29,20,26,7,26,19,16,6,2,24
514,2011,135,1427,70,1106,61,N,0,23,54,...,26,10,12,11,28,13,12,6,3,22
515,2011,135,1433,59,1425,46,N,0,20,59,...,9,15,25,5,26,4,15,2,6,20
516,2011,136,1139,60,1330,58,N,0,22,54,...,15,21,27,13,16,8,15,9,2,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,2013,146,1257,85,1181,63,N,0,29,55,...,16,21,28,11,20,11,12,4,2,23
709,2013,146,1276,79,1196,59,N,0,30,65,...,10,11,17,9,27,13,15,7,3,11
710,2013,152,1257,72,1455,68,N,0,22,49,...,20,18,24,14,22,13,11,5,1,20
711,2013,152,1276,61,1393,56,N,0,21,53,...,14,7,11,10,23,13,10,7,4,19


Povežimo sve odigrane utakmice u jednu tablicu. To su sve utakmice s kojima čemo učiti naš model.

In [556]:
all_results = pd.concat([season_compact_results, tourney_compact_results]);
all_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
113385,2011,7,1228,79,1414,65,H,0
113386,2011,7,1268,105,1370,76,H,0
113387,2011,7,1338,83,1348,75,H,0
113388,2011,7,1400,83,1298,52,H,0
113389,2011,9,1228,84,1405,45,H,0
...,...,...,...,...,...,...,...,...
1844,2013,146,1257,85,1181,63,N,0
1845,2013,146,1276,79,1196,59,N,0
1846,2013,152,1257,72,1455,68,N,0
1847,2013,152,1276,61,1393,56,N,0


Iz podataka ćemo prvo izvući varijablu RPI (Rating Percentage Index). Ako ne znate što je RPI, pogledajte https://en.wikipedia.org/wiki/Rating_percentage_index. Da bi dobili varijablu RPI trebamo najprije doći do ostale tri varijable: WP(Wining Percentage), OOWP(Opponents' Winning Percentage) i OOWP(Opponents' Opponents' Winning Percentage). Najprije ćemo izvući varijablu WP.

In [557]:
wins = all_results.groupby(['wteam', 'season'], as_index=False)['wscore'].count()
wins.rename(columns = {'wteam':'team', 'wscore':'wins'}, inplace = True)
wins

loses = all_results.groupby(['lteam', 'season'], as_index=False)['lscore'].count()
loses.rename(columns = {'lteam':'team', 'lscore':'loses'}, inplace = True)
loses

,team,season,loses
0,1101,2014,19
1,1102,2011,14
2,1102,2012,16
3,1102,2013,13
4,1102,2014,18
...,...,...,...
1382,1463,2014,13
1383,1464,2011,21
1384,1464,2012,15
1385,1464,2013,15


In [558]:
variables_per_team = pd.merge(wins, loses, on=['team', 'season'])
variables_per_team['WP'] = variables_per_team['wins'] / (variables_per_team['wins'] + variables_per_team['loses'])
variables_per_team

,team,season,wins,loses,WP
0,1101,2014,2,19,0.095238
1,1102,2011,14,14,0.500000
2,1102,2012,11,16,0.407407
3,1102,2013,15,13,0.535714
4,1102,2014,10,18,0.357143
...,...,...,...,...,...
1381,1463,2014,14,13,0.518519
1382,1464,2011,7,21,0.250000
1383,1464,2012,14,15,0.482759
1384,1464,2013,14,15,0.482759


Sada imamo WP za svaki tim i za svaku sezonu koju je tim igrao. Izračunajmo sada OWP.

In [559]:
for season in range(2011,2015):
    curr_season = all_results[all_results.season == season]
    for i, team in teams.iterrows():
        all_games_by_team = curr_season[(curr_season.wteam == team['team_id']) | (curr_season.lteam == team['team_id'])]
        summ = 0
        for j, game_by_team in all_games_by_team.iterrows():
            if(game_by_team['wteam'] == team['team_id']):
                oponent = game_by_team['lteam']
            else:
                oponent = game_by_team['wteam']
                
            games_without_team = curr_season[~curr_season.isin(all_games_by_team)].dropna(how = 'all')
            oponent_wins = len(games_without_team[games_without_team.wteam == oponent].index)
            oponent_loses = len(games_without_team[games_without_team.lteam == oponent].index)
            WP = oponent_wins / (oponent_wins + oponent_loses)
            summ += WP
            
        if ((variables_per_team['team'] == team['team_id']) & (variables_per_team['season'] == season)).any():
            variables_per_team.loc[(variables_per_team['team'] == team['team_id']) 
                        & (variables_per_team['season'] == season), ['OWP']] = summ / len(all_games_by_team.index)
            
variables_per_team

,team,season,wins,loses,WP,OWP
0,1101,2014,2,19,0.095238,0.460338
1,1102,2011,14,14,0.500000,0.551782
2,1102,2012,11,16,0.407407,0.552012
3,1102,2013,15,13,0.535714,0.555456
4,1102,2014,10,18,0.357143,0.492970
...,...,...,...,...,...,...
1381,1463,2014,14,13,0.518519,0.500480
1382,1464,2011,7,21,0.250000,0.502103
1383,1464,2012,14,15,0.482759,0.483435
1384,1464,2013,14,15,0.482759,0.480199


Nakon dobivanja OWP-a, relativno je lagano doći do treće varijable, OOWP-a.

In [560]:
for season in range(2011,2015):
    curr_season = all_results[all_results.season == season]
    for i, team in teams.iterrows():
        all_games_by_team = curr_season[(curr_season.wteam == team['team_id']) | (curr_season.lteam == team['team_id'])]
        summ = 0
        count = 0
        for j, game_by_team in all_games_by_team.iterrows():
            if(game_by_team['wteam'] == team['team_id']):
                oponent = game_by_team['lteam']
            else:
                oponent = game_by_team['wteam']
                
            if ((variables_per_team['team'] == oponent) & (variables_per_team['season'] == season)).any():
                OWP = variables_per_team[(variables_per_team.team == oponent) 
                        & (variables_per_team.season == season)]['OWP']
                summ += float(OWP)
                count += 1
        
        if ((variables_per_team['team'] == team['team_id']) & (variables_per_team['season'] == season)).any():
            variables_per_team.loc[(variables_per_team['team'] == team['team_id']) 
                        & (variables_per_team['season'] == season), ['OOWP']] = summ / count
            
variables_per_team

,team,season,wins,loses,WP,OWP,OOWP
0,1101,2014,2,19,0.095238,0.460338,0.474180
1,1102,2011,14,14,0.500000,0.551782,0.526151
2,1102,2012,11,16,0.407407,0.552012,0.513948
3,1102,2013,15,13,0.535714,0.555456,0.535908
4,1102,2014,10,18,0.357143,0.492970,0.515688
...,...,...,...,...,...,...,...
1381,1463,2014,14,13,0.518519,0.500480,0.479360
1382,1464,2011,7,21,0.250000,0.502103,0.509835
1383,1464,2012,14,15,0.482759,0.483435,0.497120
1384,1464,2013,14,15,0.482759,0.480199,0.493816


Nakon što smo dobili sve tri varijable potrebne za izgradnju RPI-a, jednostavnim uvrštavanjem u formulu dobivamo RPI za svaki tim u svakoj sezoni.

In [561]:
variables_per_team['RPI'] = (variables_per_team['WP'] * 0.25) + (variables_per_team['OWP'] * 0.50) + (variables_per_team['OOWP'] * 0.25)
variables_per_team

,team,season,wins,loses,WP,OWP,OOWP,RPI
0,1101,2014,2,19,0.095238,0.460338,0.474180,0.372523
1,1102,2011,14,14,0.500000,0.551782,0.526151,0.532429
2,1102,2012,11,16,0.407407,0.552012,0.513948,0.506345
3,1102,2013,15,13,0.535714,0.555456,0.535908,0.545634
4,1102,2014,10,18,0.357143,0.492970,0.515688,0.464693
...,...,...,...,...,...,...,...,...
1381,1463,2014,14,13,0.518519,0.500480,0.479360,0.499710
1382,1464,2011,7,21,0.250000,0.502103,0.509835,0.441010
1383,1464,2012,14,15,0.482759,0.483435,0.497120,0.486687
1384,1464,2013,14,15,0.482759,0.480199,0.493816,0.484243


Pogledajmo koje sve atribute sadrži tablica season_detailed_results.

In [562]:
print(season_detailed_results.columns.tolist())

['season', 'daynum', 'wteam', 'wscore', 'lteam', 'lscore', 'wloc', 'numot', 'wfgm', 'wfga', 'wfgm3', 'wfga3', 'wftm', 'wfta', 'wor', 'wdr', 'wast', 'wto', 'wstl', 'wblk', 'wpf', 'lfgm', 'lfga', 'lfgm3', 'lfga3', 'lftm', 'lfta', 'lor', 'ldr', 'last', 'lto', 'lstl', 'lblk', 'lpf']


Povežimo sada season_detailed_results i tourney_detailed_results kako bi dobili sve odigrane utakmice u jednoj tablici. Slično smo već radili, ali ove dvije tablice sadrže više podataka koji će nam biti potrebni.

In [563]:
all_detailed_results = pd.concat([season_detailed_results, tourney_detailed_results]);
all_detailed_results

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
39337,2011,7,1228,79,1414,65,H,0,29,58,...,20,14,22,11,26,12,14,7,1,21
39338,2011,7,1268,105,1370,76,H,0,35,62,...,16,17,22,18,16,11,27,18,1,31
39339,2011,7,1338,83,1348,75,H,0,29,64,...,32,13,16,10,21,16,17,7,6,25
39340,2011,7,1400,83,1298,52,H,0,30,59,...,23,10,18,8,22,8,16,5,5,24
39341,2011,9,1228,84,1405,45,H,0,37,64,...,12,10,19,6,17,9,23,6,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,2013,146,1257,85,1181,63,N,0,29,55,...,16,21,28,11,20,11,12,4,2,23
709,2013,146,1276,79,1196,59,N,0,30,65,...,10,11,17,9,27,13,15,7,3,11
710,2013,152,1257,72,1455,68,N,0,22,49,...,20,18,24,14,22,13,11,5,1,20
711,2013,152,1276,61,1393,56,N,0,21,53,...,14,7,11,10,23,13,10,7,4,19


Iz gore navedene tablice izvlačimo **ukupan broj poena, prosjek ukradenih lopti po utakmici, prosjek blokova po utakmici i postotak pogođenog šuta** za svaki tim i svaku sezonu. Sve nove varijable stavljamo u tablicu variables_per_team.

In [ ]:
team_arr = []
season_arr = []
pts_arr = []
st_arr = []
blks_arr = []
fg_arr = []
for i, row in variables_per_team.iterrows():
    res_in_season = all_detailed_results[(all_detailed_results.season == row['season']) 
                                 & ((all_detailed_results.wteam == row['team']) | (all_detailed_results.lteam == row['team']))]
    wins = res_in_season[res_in_season.wteam == row['team']]
    loses = res_in_season[res_in_season.lteam == row['team']]
    
    PTS = wins['wscore'].sum() + loses['lscore'].sum()
    ST = round((wins['wstl'].sum() + loses['lstl'].sum()) / (wins['wstl'].count() + loses['lstl'].count()), 1)
    BLKS = round((wins['wblk'].sum() + loses['lblk'].sum()) / (wins['wblk'].count() + loses['lblk'].count()), 1)
    FG = round((wins['wfgm'].sum() + loses['lfgm'].sum()) / (wins['wfga'].sum() + loses['lfga'].sum()) * 100, 1)

    team_arr.append(int(row['team']))
    season_arr.append(int(row['season']))
    pts_arr.append(PTS)
    st_arr.append(ST)
    blks_arr.append(BLKS)
    fg_arr.append(FG)
    
df = pd.DataFrame({
    'team' : team_arr,
    'season' : season_arr,
    'PTS': pts_arr,
    'ST' : st_arr,
    'BLKS' : blks_arr,
    'FG' : fg_arr
})

variables_per_team = pd.merge(variables_per_team, df, on=['team', 'season'])
variables_per_team

Nakon što smo dobili RPI, moramo timove rangirati po tom RPI-u, jer nam je RPI rang potreban za model. Dobivamo novu varijablu **RPI rank**.

In [ ]:
rpi_rank = {}

for year in range(2011, 2015):
    x = variables_per_team[variables_per_team.season == year][['team', 'season', 'RPI']].sort_values(by='RPI', ascending=False)
    x["RPIrank"] = x["RPI"].rank(ascending=0)
    rpi_rank[year] = x

rpi_rank[2011]

Nakon što smo dobili sve potrebne varijable za naš model (osim BPI-a za kojeg nisam uspio pronaći adekvatne izvore i informacije) krećemo pripremati naš skup podataka za treniranje. Za svaku utakmicu moramo izračunati razliku gore dobivenih varijabli. Stupci s prefiksom w označavaju varijable pobjednika, a stupci s prefiksom l označavaju varijable gubitnika.

In [ ]:
train_data = all_results.copy()
train_data = train_data.drop(columns=['daynum', 'wscore', 'lscore', 'wloc', 'numot'])

train_data = pd.merge(train_data, variables_per_team, left_on=['season', 'wteam'], right_on=['season', 'team'])
train_data = train_data.drop(columns=['team', 'wins', 'loses', 'OWP', 'OOWP', 'RPI', 'WP'])
train_data = train_data.rename(columns={'WP':'wwp', 'RPI': 'wrpi', 'PTS':'wpts', 'ST':'wst', 'BLKS':'wblks', 'FG':'wfg'})

train_data = pd.merge(train_data, variables_per_team, left_on=['season', 'lteam'], right_on=['season', 'team'])
train_data = train_data.drop(columns=['team', 'wins', 'loses', 'OWP', 'OOWP', 'RPI', 'WP'])
train_data = train_data.rename(columns={'WP':'lwp', 'RPI': 'lrpi', 'PTS':'lpts', 'ST':'lst', 'BLKS':'lblks', 'FG':'lfg'})

rpi_rank_all = pd.concat([rpi_rank[2011], rpi_rank[2012], rpi_rank[2013], rpi_rank[2014]])

train_data = pd.merge(train_data, rpi_rank_all, left_on=['season', 'wteam'], right_on=['season', 'team'])
train_data = train_data.drop(columns=['team', 'RPI'])
train_data = train_data.rename(columns={'RPIrank':'wRpiRank'})

train_data = pd.merge(train_data, rpi_rank_all, left_on=['season', 'lteam'], right_on=['season', 'team'])
train_data = train_data.drop(columns=['team', 'RPI'])
train_data = train_data.rename(columns={'RPIrank':'lRpiRank'})
    
    
train_data

**Win percentage** je varijabla koju računamo kao broj utakmica koje je favorit dobio kroz broj ukupno odigranih utakmica. Sada za svaku utakmicu moramo izračunati Win percentage. Računamo ga tako da skup podataka grupiramo po razlici između varijabli. Ukupan broj utakmica je tada count() od svih utakmica koje imaju jednaku razliku između varijabli (neovisno bila ona pozitivna ili negativna), a broj pobjeda favorita je count() od svih utakmica koje imaju jednaku razliku između varijabli, ali moraju imati i jednak preznak. Nakon tog izračuna dobivamo Win percentage za pojedinu varijablu (u ovom donjem slučaju za varijablu RPI), te nam on koristi za treniranje našeg modela.

In [ ]:
rpi_train = train_data[['wRpiRank','lRpiRank']]
rpi_train.loc[:,'rpiDiff'] = rpi_train['lRpiRank'] - rpi_train['wRpiRank']
rpi_train = rpi_train.groupby('rpiDiff', as_index=False)['wRpiRank'].count()

rpi_train_neg = rpi_train[rpi_train.rpiDiff < 0]
rpi_train_neg = rpi_train_neg.rename(columns={'wRpiRank':'count-'})
rpi_train_pos = rpi_train[rpi_train.rpiDiff > 0]
rpi_train_pos = rpi_train_pos.rename(columns={'wRpiRank':'count+'})

rpi_train_neg['rpiDiff'] = abs(rpi_train_neg['rpiDiff'])

rpi_train = pd.merge(rpi_train_neg, rpi_train_pos, how='outer', on='rpiDiff')
rpi_train.isna().sum()

In [ ]:
tmp = rpi_train.copy()
rpi_train.loc[tmp['count-'].isna(), 'count-'] = 0
rpi_train.isna().sum()

tmp = rpi_train.copy()
rpi_train['win_perc'] = tmp['count-'] / (tmp['count-'] + tmp['count+'])
rpi_train.sort_values(by='rpiDiff')

Nakon dobivanja Win percentagea imamo sve potrebno da naučimo naš model za varijablu RPE difference. Graf koji sam dobio nakon što je model **rpi_model** naučen prikazan je ispod.

In [ ]:
rpi_model = np.poly1d(np.polyfit(rpi_train.rpiDiff, rpi_train.win_perc, 2))
line = np.linspace(1, 330, 500)

plt.scatter(rpi_train.rpiDiff, rpi_train.win_perc, label='Win %')
plt.plot(line, rpi_model(line), c='r', label='Poly (win %)')
plt.xlabel('RPI Rank Difference')
plt.ylabel('Wining %')
plt.legend(loc='best')
plt.title('RPI Difference')
plt.show()

Na gotovo identičan način dobivamo win percentage i za ostale varijable, te ih uvrštavamo u funkciju regresije i učimo naše modele

In [ ]:
pts_train = train_data[['wpts', 'lpts']]
tmp = pts_train.copy()
pts_train.loc[:,'pts_diff'] = tmp.loc[:,'wpts'] - tmp.loc[:,'lpts']
pts_train = pts_train.groupby('pts_diff', as_index=False)['wpts'].count().rename(columns={'wpts': 'count'})

pts_train_neg = pts_train[pts_train.pts_diff < 0]
pts_train_neg = pts_train_neg.rename(columns={'count':'count-'})
pts_train_pos = pts_train[pts_train.pts_diff > 0]
pts_train_pos = pts_train_pos.rename(columns={'count':'count+'})

pts_train_neg['pts_diff'] = abs(pts_train_neg['pts_diff'])

pts_train = pd.merge(pts_train_neg, pts_train_pos, how='outer', on='pts_diff')

pts_train.loc[pts_train['count-'].isna(), 'count-'] = 0
pts_train.loc[pts_train['count+'].isna(), 'count+'] = 0
pts_train

tmp = pts_train.copy()
pts_train['win_perc'] = tmp['count+'] / (tmp['count+'] + tmp['count-'])
pts_train

In [ ]:
pts_model = np.poly1d(np.polyfit(pts_train.pts_diff, pts_train.win_perc, 2))
line = np.linspace(1, 1300, 500)

plt.scatter(pts_train.pts_diff, pts_train.win_perc, label='Win %')
plt.plot(line, pts_model(line), c='r', label='Poly (win %)')
plt.xlabel('Points Difference')
plt.ylabel('Wining %')
plt.legend(loc='best')
plt.title('Points Difference')
plt.show()

In [ ]:
st_train = train_data[['wst', 'lst']]
st_train.loc[:,'st_diff'] = st_train.loc[:,'wst'] - st_train.loc[:, 'lst']
st_train = st_train.groupby('st_diff', as_index=False)['wst'].count()

st_train_neg = st_train[st_train.st_diff < 0]
st_train_neg = st_train_neg.rename(columns={'wst':'count-'})
st_train_pos = st_train[st_train.st_diff > 0]
st_train_pos = st_train_pos.rename(columns={'wst':'count+'})

st_train_neg['st_diff'] = abs(st_train['st_diff'])

st_train = pd.merge(st_train_neg, st_train_pos, how='outer', on='st_diff')

st_train.loc[st_train['count-'].isna(), 'count-'] = 0
st_train.loc[st_train['count+'].isna(), 'count+'] = 0
st_train

tmp = st_train.copy()
st_train['win_perc'] = tmp['count+'] / (tmp['count+'] + tmp['count-'])
st_train.sort_values(by='st_diff')

In [ ]:
st_model = np.poly1d(np.polyfit(st_train.st_diff, st_train.win_perc, 2))
line = np.linspace(0, 7, 500)

plt.scatter(st_train.st_diff, st_train.win_perc, label='Win %')
plt.plot(line, st_model(line), c='r', label='Poly (win %)')
plt.xlabel('Steals Difference')
plt.ylabel('Wining %')
plt.legend(loc='best')
plt.title('Steals Difference')
plt.show()

In [ ]:
train = train_data[['wblks', 'lblks']]
train.loc[:,'blks_diff'] = train.loc[:,'wblks'] - train.loc[:, 'lblks']
train = train.groupby('blks_diff', as_index=False)['wblks'].count()

train_neg = train[train.blks_diff < 0]
train_neg = train_neg.rename(columns={'wblks':'count-'})
train_pos = train[train.blks_diff > 0]
train_pos = train_pos.rename(columns={'wblks':'count+'})

train_neg['blks_diff'] = abs(train['blks_diff'])

train = pd.merge(train_neg, train_pos, how='outer', on='blks_diff')

train.loc[train['count-'].isna(), 'count-'] = 0
train.loc[train['count+'].isna(), 'count+'] = 0
train

tmp = train.copy()
train['win_perc'] = tmp['count+'] / (tmp['count+'] + tmp['count-'])
train.sort_values(by='blks_diff')

In [ ]:
blks_model = np.poly1d(np.polyfit(train.blks_diff, train.win_perc, 2))
line = np.linspace(0, 7, 500)

plt.scatter(train.blks_diff, train.win_perc, label='Win %')
plt.plot(line, blks_model(line), c='r', label='Poly (win %)')
plt.xlabel('Blocks Difference')
plt.ylabel('Wining %')
plt.legend(loc='best')
plt.title('Blocks Difference')
plt.show()

In [ ]:
train = train_data[['wfg', 'lfg']]
train.loc[:,'fg_diff'] = train.loc[:,'wfg'] - train.loc[:, 'lfg']
train = train.groupby('fg_diff', as_index=False)['wfg'].count()

train_neg = train[train.fg_diff < 0]
train_neg = train_neg.rename(columns={'wfg':'count-'})
train_pos = train[train.fg_diff > 0]
train_pos = train_pos.rename(columns={'wfg':'count+'})

train_neg['fg_diff'] = abs(train['fg_diff'])

train = pd.merge(train_neg, train_pos, how='outer', on='fg_diff')

train.loc[train['count-'].isna(), 'count-'] = 0
train.loc[train['count+'].isna(), 'count+'] = 0
train

tmp = train.copy()
train['win_perc'] = tmp['count+'] / (tmp['count+'] + tmp['count-'])
train.sort_values(by='fg_diff')

In [ ]:
fg_model = np.poly1d(np.polyfit(train.fg_diff, train.win_perc, 2))
line = np.linspace(0, 13, 500)

plt.scatter(train.fg_diff, train.win_perc, label='Win %')
plt.plot(line, fg_model(line), c='r', label='Poly (win %)')
plt.xlabel('Field Goal Percent Difference')
plt.ylabel('Wining %')
plt.legend(loc='best')
plt.title('Field Goal Percent Difference')
plt.show()

Sada imamo 5 modela: **rpi_model, pts_model, st_model, blks_model i fg_model**. Sada možemo izabrati neki novi set podataka i na temelju dobivenih modela obaviti predikciju za utakmice u tom novom skupu podataka. Odabrao sam skup podataka koji prikazuje turnirske utakmice u sezoni 2014.

In [ ]:
tourney_2014_results = pd.read_csv('./data/proj2/tourney_detailed_results.csv')
tourney_2014_results = tourney_2014_results[tourney_2014_results.season == 2014]
tourney_2014_results

In [ ]:
tourney_2014_results = tourney_2014_results[['wteam','lteam']]
tourney_2014_results

Sada dohvaćamo podatke o već izvedenim varijablama iz sezone 2014 te ćemo pomoću njih dobivati našu predikciju.

In [ ]:
season_2014_data = train_data[train_data.season == 2014]

season_2014_data_1 = season_2014_data[['wteam', 'wpts', 'wst', 'wblks', 'wfg', 'wRpiRank']]
season_2014_data_1 = season_2014_data_1.rename(columns={'wteam':'team', 'wpts':'pts', 'wst':'st', 'wblks':'blks', 
                                                'wfg':'fg', 'wRpiRank':'rpiRank'})
season_2014_data_1

season_2014_data_2 = season_2014_data[['lteam', 'lpts', 'lst', 'lblks', 'lfg', 'lRpiRank']]
season_2014_data_2 = season_2014_data_2.rename(columns={'lteam':'team', 'lpts':'pts', 'lst':'st', 'lblks':'blks', 
                                                'lfg':'fg', 'lRpiRank':'rpiRank'})
season_2014_data_2

season_2014_data = pd.concat([season_2014_data_1, season_2014_data_2]).drop_duplicates()
season_2014_data

Nakon toga za svaku utakmicu u turniru 2014-e moramo pronaći razliku varijabli jer ćemo razliku varijabli davati našim modelima i na temelju toga dobiti predikciju. **Tourney_2014_results** prikazuje sve potrebne podatke koji su nam potrebni da bi napravili predikciju za utakmice turnira 2014.

In [ ]:
tmp = tourney_2014_results.copy()

for i, game in tmp.iterrows():
    if not (season_2014_data[season_2014_data.team == game['wteam']]).empty:
        if not (season_2014_data[season_2014_data.team == game['lteam']]).empty:
       
            tourney_2014_results.loc[(tourney_2014_results.wteam == game['wteam']) & (tourney_2014_results.lteam == game['lteam']), 
                                     'rpiDiff'] = int(season_2014_data[season_2014_data.team == game['lteam']]['rpiRank']) - int(season_2014_data[season_2014_data.team == game['wteam']]['rpiRank'])

            tourney_2014_results.loc[(tourney_2014_results.wteam == game['wteam']) & (tourney_2014_results.lteam == game['lteam']), 
                                     'ptsDiff'] = int(season_2014_data[season_2014_data.team == game['wteam']]['pts']) - int(season_2014_data[season_2014_data.team == game['lteam']]['pts'])

            tourney_2014_results.loc[(tourney_2014_results.wteam == game['wteam']) & (tourney_2014_results.lteam == game['lteam']), 
                                     'stDiff'] = float(season_2014_data[season_2014_data.team == game['wteam']]['st']) - float(season_2014_data[season_2014_data.team == game['lteam']]['st'])

            tourney_2014_results.loc[(tourney_2014_results.wteam == game['wteam']) & (tourney_2014_results.lteam == game['lteam']), 
                                     'blksDiff'] = float(season_2014_data[season_2014_data.team == game['wteam']]['blks']) - float(season_2014_data[season_2014_data.team == game['lteam']]['blks'])

            tourney_2014_results.loc[(tourney_2014_results.wteam == game['wteam']) & (tourney_2014_results.lteam == game['lteam']), 
                                     'fgDiff'] = float(season_2014_data[season_2014_data.team == game['wteam']]['fg']) - float(season_2014_data[season_2014_data.team == game['lteam']]['fg'])

tourney_2014_results = tourney_2014_results.loc[tourney_2014_results.rpiDiff.notnull(), :]
tourney_2014_results

Sada za svaku utakmicu u tourney_2014_results radimo predikciju po svakoj varijabli pomoću gore dobivenih modela. Kada izračunamo predikciju po svakoj varijabli, jednostavnim računanjem aritmetičke sredine svih predikcija dobivamo win percentage tima koji je pobijedio. Ako nam je win percentage ispao veći od 50%, to znači da je naša predikcija točna i da smo pogodili pobjednika utakmice.

In [ ]:
tmp = tourney_2014_results.copy()

for i, result in tmp.iterrows():
    x = float(result['rpiDiff'])
    rpi = 0
    if(x <= 0):
        rpi = 1 - rpi_model(x)
    else:
        rpi = rpi_model(x)
        
        
    x = float(result['ptsDiff'])
    pts = 0
    if(x <= 0):
        pts = 1 - pts_model(x)
    else:
        pts = pts_model(x)
        
    
    x = float(result['stDiff'])
    st = 0
    if(x <= 0):
        st = 1 - st_model(x)
    else:
        st = st_model(x)
        
    
    x = float(result['blksDiff'])
    blks = 0
    if(x <= 0):
        blks = 1 - blks_model(x)
    else:
        blks = blks_model(x)
        
        
    x = float(result['fgDiff'])
    fg = 0
    if(x <= 0):
        fg = 1 - fg_model(x)
    else:
        fg = fg_model(x)
        
        
    pred = (rpi + pts + st + blks + fg) / 5
    
    tourney_2014_results.loc[(tourney_2014_results.wteam == result['wteam']) & (tourney_2014_results.lteam == result['lteam']), 
                             'pred'] = pred
    
tourney_2014_results

Nakon malo uglađivanja dobivamo finalnu tablicu i finalni postotak pogođenog ishoda utakmice. Finalni postotak pogođenih ishoda je veći nego u članku iz ne znam kojeg razloga, možda je moja greška, možda je upotreba drugog skupa podataka za predikciju.

In [ ]:
final = tourney_2014_results[['wteam', 'lteam', 'pred']]
final = pd.merge(final, teams, left_on='wteam', right_on='team_id').drop(columns=['wteam', 'team_id']).rename(columns={'team_name':'wteam'})
final = pd.merge(final, teams, left_on='lteam', right_on='team_id').drop(columns=['lteam', 'team_id']).rename(columns={'team_name':'lteam'})
final = final.iloc[:, [1,2,0]]
final.loc[final.pred > 0.5, 'correct'] = 1
final.loc[final.pred <= 0.5, 'correct'] = 0
pd.set_option('display.max_rows', None)
final

In [ ]:
pd.set_option('display.max_rows', 10)
print('Pogođene utakmice: ', len(tourney_2014_results[tourney_2014_results.pred > 0.5].index))
print('Odigrane utakmice: ', len(tourney_2014_results.index))
print('Postotak pogođenog: ', round(len(tourney_2014_results[tourney_2014_results.pred > 0.5].index) / len(tourney_2014_results.index) * 100,2), '%')